In [85]:
import numpy as np
import tensorflow.compat.v1 as tf
from tensorflow.keras import layers, Sequential, Input
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import generator

config = tf.ConfigProto(device_count = {'DML': 0})
sess = tf.Session(config=config)

In [86]:
def generate_dataset():
    gen = generator.Generator(path_s="data/initial kernels/Kernel_Carbon_Adsorption.npy",
                        path_d="data/initial kernels/Kernel_Carbon_Desorption.npy",
                        path_p_d="data/initial kernels/Pressure_Carbon.npy",
                        path_p_s="data/initial kernels/Pressure_Carbon.npy",
                        path_a="data/initial kernels/Size_Kernel_Carbon_Adsorption.npy"
                )
    gen.generate_data_set()

def show_dataset():
        dataset = hkl.load('data/datasets/carbon3.hkl')
        i = 8
        plt.plot(dataset[i]["isotherm"], marker=".")
        plt.plot(dataset[i]["pore_distribution"], marker=".")
        plt.show()
        print(len(dataset))

In [87]:
def plot_loss(history):
    plt.plot(history.history['loss'], label='loss', marker=".")
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Error [MPG]')
    plt.legend()
    plt.grid(True)
    plt.yscale("log")
    plt.show()

In [138]:
min_exp_pressure_i = 48
max_exp_pressure_i = 546
with open('data/datasets/carbon_2_equal_peaks.npz', 'rb') as f:
        dataset = np.load(f)
        isotherm_data = dataset["isotherm_data"]
        pore_distribution_data = dataset["pore_distribution_data"]
x = np.empty((isotherm_data.shape[0], (-min_exp_pressure_i + max_exp_pressure_i)))
y = np.empty(pore_distribution_data.shape)
for i in range(len(isotherm_data)):
    isotherm = isotherm_data[i][min_exp_pressure_i:max_exp_pressure_i]
    isotherm -= min(isotherm)
    isotherm /= max(isotherm)
    pore_distribution = pore_distribution_data[i] - min(pore_distribution_data[i])
    pore_distribution /= max(pore_distribution)
    x[i] = isotherm
    y[i] = pore_distribution
x, y = shuffle(x, y)

In [137]:
pressures = np.load("data/initial kernels/Pressure_Carbon.npy")
pressures[546]

0.998000025749207

In [108]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [117]:
norm_layer = layers.Normalization()
norm_layer.adapt(x)

In [121]:
### DENSE NET
model = Sequential(
    [
        Input(shape=len(x_train[0], )),
        norm_layer,
        layers.Dense(400, activation='relu'),
        layers.Dense(400, activation='relu'),
        layers.Dense(400, activation='relu'),
        layers.Dense(400, activation='relu'),
        layers.Dense(400, activation='relu'),
        layers.Dense(400, activation='relu'),
        layers.Dense(400, activation='relu'),
        layers.Dense(len(y_train[0])) #  activation=tf.math.abs
    ]
)
### CONV NET 
# x_train=np.array(x_train)
# x_test=np.array(x_test)
# x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
# x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))
# def make_model(input_shape, norm_layer):
#     input_layer = tf.keras.layers.Input(input_shape)
# 
#     conv1 = tf.keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
#     conv1 = tf.keras.layers.BatchNormalization()(conv1)
#     conv1 = tf.keras.layers.ReLU()(conv1)
# 
#     conv2 = tf.keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
#     conv2 = tf.keras.layers.BatchNormalization()(conv2)
#     conv2 = tf.keras.layers.ReLU()(conv2)
# 
#     conv3 = tf.keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
#     conv3 = tf.keras.layers.BatchNormalization()(conv3)
#     conv3 = tf.keras.layers.ReLU()(conv3)
# 
#     gap = tf.keras.layers.GlobalAveragePooling1D()(conv3)
# 
#     output_layer = tf.keras.layers.Dense(len(y_train[0]))(gap)
# 
#     return tf.keras.models.Model(inputs=input_layer, outputs=output_layer)
# 
# norm_layer = tf.keras.layers.Normalization(axis=None)
# norm_layer.adapt(np.array(x))
# model = make_model(input_shape=np.array(x_train).shape[1:], norm_layer=norm_layer)

In [122]:
model.compile(loss='mean_squared_error', optimizer='Adam')

In [139]:
mcp_save = tf.keras.callbacks.ModelCheckpoint(filepath='data/models/carbon2_0-1_normalization.keras', save_best_only=True,
                                           monitor='val_loss', mode='min', verbose=1, save_weights_only=False,
                                           save_freq='epoch')
reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                                   patience=100, verbose=1, mode='auto')
# from keras import backend as K
# K.set_value(model.optimizer.learning_rate, 0.00001)
history = model.fit(np.array(x_train), np.array(y_train),
                    epochs=1000, batch_size=7500, shuffle=True,
                    validation_data=(np.array(x_test), np.array(y_test)), callbacks=[mcp_save, reduce_lr_loss])

Epoch 1/1000
 9/10 [==========================>...] - ETA: 0s - loss: 1.7009e-04
Epoch 1: val_loss improved from inf to 0.00019, saving model to data/models\carbon2_0-1_normalization.keras
10/10 [==============================] - 1s 79ms/step - loss: 1.7041e-04 - val_loss: 1.8605e-04 - lr: 6.2500e-05
Epoch 2/1000
 8/10 [=======================>......] - ETA: 0s - loss: 1.7363e-04

KeyboardInterrupt: 

In [126]:
plot_loss(history)

In [125]:
# load model
#model = tf.keras.models.load_model('data/models/carbon_two_equal_peaks.keras', custom_objects={'abs': tf.math.abs})
#
prediction = model.predict(np.array(x_train))
print("prediction shape:", prediction.shape)

2344/2344 [==============================] - 11s 5ms/step
prediction shape: (75000, 130)


In [152]:
pore_widths = np.load("data/initial kernels/Size_Kernel_Carbon_Adsorption.npy")
NX, NY = 4 , 5
figure, axis = plt.subplots(NX, NY)
for i in range(NX):
    for j in range(NY):
        k = np.random.randint(0, len(prediction))
        ### Distribution and Prediction 
        axis[i, j].plot(pore_widths, prediction[k], marker=".", label=f"Prediction") 
        axis[i, j].plot(pore_widths, y_train[k], marker=".", label="Real")
        ###
        ### Distribution and Isotherm 
        # axis[i, j].plot(pore_widths/max(pore_widths)*len(x_train[k]), y_train[k], marker=".", label="Distribution")
        # axis[i, j].plot(x_train[k], marker=".", label="Isotherm")
        ###
        
        axis[i, j].set_title(f"№ {k}")
        axis[i, j].title.set_size(10)
plt.subplots_adjust(hspace=0.6, right=0.95, left=0.05, bottom=0.05, top=0.95)
plt.legend()
plt.show()

In [ ]:
import generator
gen = generator.Generator(path_s="data/initial kernels/Kernel_Carbon_Adsorption.npy",
                        path_d="data/initial kernels/Kernel_Carbon_Desorption.npy",
                        path_p_d="data/initial kernels/Pressure_Carbon.npy",
                        path_p_s="data/initial kernels/Pressure_Carbon.npy",
                        path_a="data/initial kernels/Size_Kernel_Carbon_Adsorption.npy"
                )
gen.pore_distribution = prediction[i]
gen.calculate_calculate_isotherms_right()
plt.plot(gen.pressures_s[min_exp_pressure_i:max_exp_pressure_i], x_train[i], marker=".", label="Real")
plt.plot(gen.pressures_s[min_exp_pressure_i:max_exp_pressure_i], gen.n_s[min_exp_pressure_i:max_exp_pressure_i], marker=".", label="Net")
plt.legend()
plt.show()

In [ ]:
model.save('data/models/carbon_two_equal_peaks_conv.keras')